In [1]:
import scipy.io
import numpy as np
import os

In [30]:
import sys
import warnings
import time
from tqdm import tqdm
from math import sqrt

import numpy as np
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from hyperimpute.plugins.utils.metrics import RMSE
from hyperimpute.plugins.utils.simulate import simulate_nan

import xgboost as xgb


from sklearn.metrics import mean_squared_error
import numpy as np


In [3]:
from hyperimpute.plugins.imputers import Imputers, ImputerPlugin

imputers = Imputers()

In [4]:
data_dir = r"/home/er647/data/fwal-data"

In [5]:
def load_ASU_dataset(data_dir, dataset):
	mat = scipy.io.loadmat(os.path.join(data_dir, "ASU_datasets", f"{dataset}.mat"))
	X = mat['X']
	y = np.squeeze(mat['Y'])
	X = X.astype(np.float64)
	y = y.astype(np.int64)

	if y.min() == 1 and y.max() == len(set(y)):
		y -= 1
	
	if y.min() == -1 and y.max() == 1 and len(set(y)) == 2:
		y = (y + 1) // 2

	return X, y	


In [27]:

def load_finance(data_dir):
	data = pd.read_csv(f'{data_dir}/Finance/finance.csv')
	X = data.drop(columns=['Class'])
	Y = data['Class']
	# convert dtype of Y to int
	Y = Y.astype(int)
	return X, Y

In [31]:
# X, y = load_ASU_dataset(data_dir, 'madelon')
X, y = load_finance(data_dir)

In [32]:
if type(X)==pd.DataFrame:
    X = X.to_numpy()
if type(y)==pd.Series or type(y)==pd.DataFrame:
    y = y.to_numpy()

assert type(X)==np.ndarray
assert type(y)==np.ndarray

# Split validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train).astype(np.float32)
X_test = scaler.transform(X_test).astype(np.float32)
	

In [33]:
type(X), X.shape

(numpy.ndarray, (2664, 154))

In [23]:
type(X_train), X_train.shape


(numpy.ndarray, (2131, 154))

In [35]:
def gen_MCAR_datasets(X_valid, X_test, fraction = 0.2, seed = 0, replace_val = 0, include_train = False):
    """
    Missing data mechanism
    - fraction (float): percentage of missing values
    - seed (int): seed for reproducibility
    - replace_val: 0 or np.nan for example
    """
    np.random.seed(seed)

    missing_mask_valid = np.random.choice([0, 1], size=X_valid.shape, p=[1-fraction, fraction])
    missing_mask_test = np.random.choice([0, 1], size=X_test.shape, p=[1-fraction, fraction])


    X_valid_missing = X_valid.copy()
    X_valid_missing[missing_mask_valid==1] = replace_val

    X_test_missing = X_test.copy()
    X_test_missing[missing_mask_test==1] = replace_val
    return X_valid_missing, X_test_missing, missing_mask_valid, missing_mask_test
X_valid_missing, X_test_missing, missing_mask_valid, missing_mask_test = gen_MCAR_datasets(X_train, X_test, replace_val=np.nan)

In [37]:
np.any(np.isinf(X_test_missing) & ~np.isnan(X_test_missing))

False

In [25]:
imputers.list()

['median',
 'ice',
 'mice',
 'hyperimpute',
 'miwae',
 'most_frequent',
 'softimpute',
 'sklearn_missforest',
 'EM',
 'mean',
 'nop',
 'sinkhorn',
 'missforest',
 'gain',
 'miracle',
 'sklearn_ice']

In [38]:
benchmarks = [
    # 'miwae', # too computationally expensive
    # 'softimpute',
    # 'mean',
    # 'miracle', # too slow
    # 'sinkhorn',
    # 'sklearn_missforest',
    # 'most_frequent',
    # 'EM', # too slow
    # 'nop',
    # 'hyperimpute', # too slow
    # 'gain',
    # 'median',
    'ice', # too slow
    # 'mice', # too slow
    # 'sklearn_ice',
    # 'missforest' # too slow
]

# for benchmark in ['sklearn_ice', 'sinkhorn', 'gain', 'softimpute']:
for benchmark in benchmarks:
    start = time.time()
    imputer = imputers.get(benchmark, random_state=42)
    fitted_imputer = imputer.fit(X_train)
    fit_time = round(time.time() - start, 4)
    start = time.time()
    X_test_imputed = fitted_imputer.transform(X_test_missing).to_numpy()
    missing_mask = np.isnan(X_test_missing)  # Creating the mask where X_test_missing is NaN
    mse = mean_squared_error(X_test[missing_mask], X_test_imputed[missing_mask])  # Calculating MSE only for missing values
    print(benchmark,' | # NaNs in test set: ', np.isnan(X_test_missing).sum(), '| # NaNs in imputed set: ',np.isnan(X_test_imputed).sum(),'| fit time: ',fit_time, '| transform time (seconds): ', round(time.time() - start, 4), '| MSE: ',mse)



ice  | # NaNs in test set:  16362 | # NaNs in imputed set:  0 | fit time:  0.0937 | transform time (seconds):  130.6002 | MSE:  39.49619


In [14]:
X_test_imputed = fitted_imputer.transform(X_test_missing).to_numpy()

In [15]:
imputer = imputers.get('ice')

In [27]:
fitted_imputer = imputer.fit(X_train)

In [28]:
X_test_imputed = fitted_imputer.transform(X_test_missing).to_numpy()